In [ ]:
%matplotlib inline
import os
import os.path
import csv
import numpy
import scipy.stats
import scipy.cluster
import itertools
import math
import sys
import sklearn.cluster
import sklearn.decomposition
import sklearn.manifold
import skimage.filters
import pandas
from matplotlib import pylab, mlab
import matplotlib.cm
import matplotlib.patches
import IPython.display
# local modules (in same dir as this notebook)
import plotutils
import datautils
import syspathutils


In [ ]:
base_path = os.path.realpath(os.path.join(os.path.curdir, '..'))
data_path = os.path.join(base_path, 'samples')
syspathutils.append_to_sys_path('/home/gio/langdev/gitclones/arbimon2-jobs/.env/lib/python2.7/site-packages/')
syspathutils.append_to_sys_path('/home/gio/langdev/gitclones/arbimon2-jobs/lib/')

In [ ]:
import a2audio.rec
import a2pyutils.storage
import a2audio.segmentation
import a2pyutils.convert

In [ ]:
datasets = {x.replace('sample', '').strip('_'):os.path.join(data_path, x) for x in os.listdir(data_path)}
print '\n'.join("{} : {}".format(x, y) for x, y in datasets.items())

In [ ]:
# dataset = 'smithsonian'
# dataset = 'el_verde1'
dataset = 'sabana_seca1'
# roi_filepath = ('/home/gio/langdev/gitclones/arbimon2-jobs/synth_output/rois.txt')
dataset_path = datasets[dataset]

In [ ]:
matplotlib.rcParams['font.size'] = 18

# Some methods for formatting the plot ticks
class TickFormat:
    @staticmethod
    def am_pm(x):
        x=int(x)
        return "{}{}".format(x%12 or 12, "pm" if x > 12 else "am")

    @staticmethod
    def kHz(x, prec=1):
        return "{}kHz".format(int(x/(1000*prec))*prec)

    @staticmethod
    def kHz_1(x):
        return TickFormat.kHz(x, .1)
    
    sec = "%2ds"

    @staticmethod
    def percent(x):
        return "{}%".format(int(float(x)*100))

def show_plot(var1, var2, fmt1, fmt2, b1, b2, X=None):
    if X is None:
        X = rois_data
    pylab.figure(figsize=(10,10))
    plotutils.density_contour_plot(X[var1], X[var2], bins=[b2, b1], interpolation="nearest", log=True)
    plotutils.set_plot_axis(pylab.gca(), 'x', visible=True, label=var1)
    plotutils.set_plot_axis(pylab.gca(), 'y', visible=True, label=var2)
    plotutils.set_tick_labels(pylab.gca(), 'x', fmt1 if callable(fmt1) else getattr(TickFormat, fmt1))
    plotutils.set_tick_labels(pylab.gca(), 'y', fmt2 if callable(fmt2) else getattr(TickFormat, fmt2))

In [ ]:
areas_of_interest = pandas.read_csv(os.path.join(dataset_path, 'areas_of_interest.txt'))

for i, aoi in areas_of_interest.iterrows():
    print "subregion:: {} <= bw <= {}, {} <= y_max <= {}".format(aoi.bw_min, aoi.bw_max, aoi.y_max_min, aoi.y_max_max)

In [ ]:
rois_subdata_sample_full = pandas.read_csv(os.path.join(dataset_path, 'aoi2_sampled_rois.txt'), index_col=0)
rois_subdata_sample_sorted = rois_subdata_sample_full.sort(['rec','x'])
# rois_subdata_sample_full = rois_data_ori.loc[rois_subdata_sample_bounds.index]
print "entries:{}, cols:{}".format(len(rois_subdata_sample_full), list(rois_subdata_sample_full.columns))

In [ ]:
sabana_seca1_sample_storage = a2pyutils.storage.from_uri(dataset_path)
# region_rois_output = a2pyutils.storage.from_uri(os.path.join(dataset_path, 'aoi1_rois')

idxx = [28]
pad=50

for i, (idx, roi_row) in enumerate(rois_subdata_sample_full.iloc[idxx].iterrows()):
    pylab.figure(figsize=(21,21))
# for i, (idx, roi_row) in enumerate(rois_subdata_sample_full.iterrows()):
    print "{} of {}   idx:{} rec:{}".format(i, len(rois_subdata_sample_full), idx, roi_row['rec'])
    roi_rec = a2audio.rec.Rec(roi_row['rec'], '', sabana_seca1_sample_storage)
    spectrum, freqs, times = roi_rec.getSpectrogram()
    specDB = a2pyutils.convert.power2decibels(spectrum)
    max_v = specDB.flatten().max()
    duration = roi_rec.samples * 1.0 / roi_rec.sample_rate
    max_freq = roi_rec.sample_rate / 2.0
    px2sec = duration / specDB.shape[1]
    px2Hz = max_freq / specDB.shape[0]
    print "rec: samples:{}, samplerate: {}, duration:{}, maxfreq:{}, scale({},{})".format(roi_rec.samples, roi_rec.sample_rate,duration, max_freq, px2sec,px2Hz)
    
    x0, y1 = roi_row['x'], specDB.shape[0] - roi_row['y']
    x1, y0 = x0 + roi_row['w'], y1 - roi_row['h']
    
    t0, t1 = x0 * px2sec, x1 * px2sec
    f0, f1 = y0 * px2Hz, y1 * px2Hz
    
    roi = datautils.clip_pad_and_draw_rgb_bounds(specDB, x0, x1,y0, y1, pad, (.25,1.0,.25))
    
    pylab.title("{}\n ({}-{} s)-({}-{}Hz)\n ({}-{} s)-({}-{}Hz)".format(roi_row['rec'],t0,t1,f0,f1, 
        roi_row['t0'], roi_row['t1'], roi_row['f0'], roi_row['f1']))
    pylab.imshow(
         roi,
         interpolation="nearest",
         origin='lower'
    )

    
#     with region_rois_output.open_for_writing("roi2-{}.png".format(i)) as fout:
#         pylab.imsave(fout, 
#              roi,
#              origin='lower'
#         )

In [ ]:
idx = 13

roi_row = rois_subdata_sample_full.iloc[idx]
roi_rec = a2audio.rec.Rec(roi_row['rec'], '', sabana_seca1_sample_storage)

spec2samp = roi_rec.original.shape[0] * 1.0 / spectrum.shape[1]
pad=10
samp_x0, samp_x1 = int((x0-pad)*spec2samp), int((x1+pad)*spec2samp+.5)

roi_clip = roi_rec.original[samp_x0:samp_x1]

filter_schedule = [0,0,0,1,1,2,2,3,3,3,2,2,1,1,0,0,0]
# filter_schedule = [0,0,0,1,1,2,2,3,4,5,5,5,4,3,2,2,1,1,0,0,0]

roi_clip_bpf = numpy.concatenate([datautils.butter_bandpass_filter(roi_rec.original, f0, f1, roi_rec.sample_rate, order=1, passes=i)[samp_x0:samp_x1] for i in filter_schedule])

pylab.figure(figsize=(15,5))
pylab.suptitle("ROI #{}".format(idx))
pylab.plot(roi_clip_bpf)
pylab.figure(figsize=(10,5))
pylab.subplot(1,2,2)
pylab.specgram(roi_clip, Fs=roi_rec.sample_rate)
pylab.hlines(roi_row)
cb1=pylab.colorbar()
pylab.subplot(1,2,1)
pylab.specgram(roi_clip_bpf, Fs=roi_rec.sample_rate, vmin=cb1.vmin, vmax=cb1.vmax)

IPython.display.display(IPython.display.Audio(roi_clip_bpf, rate=roi_rec.sample_rate))

In [ ]:
reload(a2audio.rec)
sabana_seca1_sample_storage = a2pyutils.storage.from_uri(dataset_path)
# region_rois_output = a2pyutils.storage.from_uri(os.path.join(dataset_path, 'aoi1_rois')



# idxx = [1, 38]
for i, (idx, roi_row) in enumerate(rois_subdata_sample_sorted.iloc[[3]].iterrows()):
# for i, (idx, roi_row) in enumerate(rois_subdata_sample_full.iterrows()):
    roi_rec = a2audio.rec.Rec(roi_row['rec'], '', sabana_seca1_sample_storage)
    spectrum, freqs, times = roi_rec.getSpectrogram()
#     duration = roi_rec.original_sample_count * 1.0 / roi_rec.original_sample_rate
#     duration = roi_rec.samples * 1.0 / roi_rec.sample_rate
#     px2sec = duration / (roi_rec.samples / 256)
    duration = roi_rec.samples * 1.0 / roi_rec.sample_rate
    px2sec = duration / spectrum.shape[1]
    max_freq = roi_rec.sample_rate / 2.0
    px2Hz = max_freq / spectrum.shape[0]
    
    x0, y1 = roi_row['x'], spectrum.shape[0] - roi_row['y']
    x1, y0 = x0 + roi_row['w'], y1 - roi_row['h']
    
    t0, t1 = x0 * px2sec, x1 * px2sec
    f0, f1 = y0 * px2Hz, y1 * px2Hz
    
    print "{},{},{}, {}, spec/data scale : {}".format(
        roi_row['rec'],
        t0,t1,
        (times[x0],times[x1]),
        (roi_rec.samples) * 1.0 / spectrum.shape[1]
    )


In [ ]:
SR=44100
dur=60
C0 = 440 * 2**(-9/12)
def fib(n):
    sqrt5 = 5**.5
    phi = (1 + sqrt5)/2
    return (phi**(n+1) - (1-phi)**(n+1)) / sqrt5
MayorScale = lambda n: 2*(n%7) - (numpy.sign(n%7 - 2)+1)/2 + 12 * numpy.floor(n/7)
PureTone = lambda t, x : numpy.cos(t * 2 * numpy.pi * C0 * 2 ** (MayorScale(fib(numpy.floor(numpy.pi*x)) % 7)/12.0))

X = (lambda x, r, mu, sigma2: 
    (numpy.exp(-(((r*x)%1)/r - mu)**2/(2*sigma2)) +
    numpy.exp(-((x%1) - mu)**2/(2*sigma2**2))) * PureTone(x, x)
)(numpy.linspace(0, dur, SR*dur), 5, 1/(5*2.0), .005)

Xspec, freqs, times = pylab.mlab.specgram(X, NFFT=512, Fs=SR, noverlap=256)

pylab.figure(figsize=(15,5))
pylab.subplot(1,2,1)
pylab.imshow(numpy.log(Xspec), origin="bottom", aspect="auto")
pylab.subplot(1,2,2)
pylab.plot(X[0:int(SR*.1)])
IPython.display.display(IPython.display.Audio(X, rate=SR))

In [ ]:
X2SR = 48000
X2 = a2audio.rec.resample(X, float(X2SR)/float(SR) , 'sinc_best')
X2spec, freqs, times = pylab.mlab.specgram(X2, NFFT=512, Fs=X2SR, noverlap=256)

pylab.figure(figsize=(15,5))
pylab.subplot(1,2,1)
pylab.imshow(numpy.log(X2spec), origin="bottom", aspect="auto")
pylab.subplot(1,2,2)
pylab.plot(X2[0:int(X2SR*.1)])
IPython.display.display(IPython.display.Audio(X2, rate=X2SR))

In [ ]:
x1_px2sec = X.shape[0] * 1.0 / Xspec.shape[1]
x2_ps2sec = X2.shape[0] * 1.0 / X2spec.shape[1]
x1_R_0, x1_R_1 = (lambda x0, w : (x0, x0+w))(105, 100)
x2_R_0, x2_R_1 = x1_R_0 * x1_px2sec / x2_ps2sec, x1_R_1 * x1_px2sec / x2_ps2sec

vmin, vmax= numpy.log(Xspec).min(), numpy.log(Xspec).max()

pylab.figure(figsize=(15,5))
pylab.subplot(1,2,1)
pylab.imshow(numpy.log(Xspec), origin="bottom", aspect="auto", vmin=vmin,vmax=vmax)
pylab.gca().set_xlim(x1_R_0, x1_R_1)
pylab.subplot(1,2,2)
pylab.imshow(numpy.log(X2spec), origin="bottom", aspect="auto", vmin=vmin,vmax=vmax)
pylab.gca().set_xlim(x2_R_0, x2_R_1)
